In [20]:
! pip install plotly


   ---------------------------------------- 0.0/15.7 MB ? eta -:--:--
   ---------------------------------------- 0.0/15.7 MB ? eta -:--:--
   ---------------------------------------- 0.1/15.7 MB 2.2 MB/s eta 0:00:08
   - -------------------------------------- 0.4/15.7 MB 3.1 MB/s eta 0:00:05
   -- ------------------------------------- 0.9/15.7 MB 5.2 MB/s eta 0:00:03
   -- ------------------------------------- 1.1/15.7 MB 5.0 MB/s eta 0:00:03
   --- ------------------------------------ 1.3/15.7 MB 4.8 MB/s eta 0:00:04
   --- ------------------------------------ 1.3/15.7 MB 4.7 MB/s eta 0:00:04
   --- ------------------------------------ 1.3/15.7 MB 4.7 MB/s eta 0:00:04
   --- ------------------------------------ 1.3/15.7 MB 4.7 MB/s eta 0:00:04
   ---- ----------------------------------- 1.9/15.7 MB 4.4 MB/s eta 0:00:04
   ----- ---------------------------------- 2.2/15.7 MB 4.5 MB/s eta 0:00:04
   ------ --------------------------------- 2.4/15.7 MB 4.4 MB/s eta 0:00:04
   ------ ---

In [2]:
import pandas as pd 
import matplotlib.pyplot as plt
import numpy as np
import plotly.express as plx

In [12]:
df= pd.read_csv("dating_app_dataset.csv")
df.head(10)

,User ID,Age,Gender,Height,Interests,Looking For,Children,Education Level,Occupation,Swiping History,Frequency of Usage
0,1,30,Male,5.240385,"['Sports', 'Cooking', 'Hiking', 'Music', 'Movi...",Casual Dating,No,High School,Student,96,Weekly
1,2,27,Female,4.937625,"['Sports', 'Reading']",Friendship,Yes,Master's Degree,Artist,96,Monthly
2,3,29,Female,5.806296,['Sports'],Casual Dating,No,Bachelor's Degree,Social Media Influencer,64,Daily
3,4,29,Female,5.101402,['Reading'],Marriage,No,Ph.D.,Doctor,67,Daily
4,5,32,Male,5.986670,"['Sports', 'Hiking', 'Music', 'Movies', 'Readi...",Long-term Relationship,Yes,Ph.D.,Engineer,93,Monthly
5,6,25,Female,5.192045,"['Sports', 'Cooking', 'Movies', 'Music', 'Trav...",Marriage,Yes,Master's Degree,Engineer,38,Monthly
6,7,27,Female,5.725126,"['Travel', 'Cooking']",Marriage,No,Master's Degree,Student,77,Monthly
7,8,21,Male,5.108276,"['Cooking', 'Hiking', 'Movies', 'Reading', 'Mu...",Marriage,Yes,Bachelor's Degree,Teacher,87,Daily
8,9,33,Female,5.364951,"['Music', 'Reading', 'Sports', 'Cooking', 'Mov...",Friendship,Maybe,High School,Social Media Influencer,6,Monthly
9,10,26,Female,6.020815,"['Sports', 'Reading', 'Music', 'Cooking', 'Mov...",Long-term Relationship,No,Ph.D.,Teacher,8,Weekly


In [25]:
color_map = {"Male":"blue", "Female":"pink"}
df["color"] = df["Gender"].map(color_map)

In [7]:
fig = plx.histogram(df, x="Age", color="Gender", nbins=20, 
                   title="Age Distribution by Gender")
#fig.update_traces(marker_color=['Blue', 'pink'])
fig.update_layout(xaxis_title="Age", yaxis_title="Count")
fig.show()

In [27]:
education_order = ["High School", "Bachelor's Degree", "Master's Degree", "Ph.D."]
fig = plx.bar(df, x="Education Level", color="Gender", 
             category_orders={"Education Level": education_order},
             title="Education Level Distribution by Gender")

fig.update_layout(xaxis_title="Education Level", yaxis_title="Count")
fig.show()


In [32]:
fig = plx.bar(df, x="Frequency of Usage", 
             title="Frequency of App Usage Distribution")
fig.update_layout(xaxis_title="Frequency of Usage", 
                  yaxis_title="Count")
fig.show()

Separate male and female profiles

In [28]:
male_p = df[df['Gender'] == 'Male']
female_p= df[df['Gender']== 'Female']

Calculating match score

In [40]:
def rel_Score(profile1,profile2):
    #Interests
    interest1= set(eval(profile1['Interests']))
    interest2= set(eval(profile2['Interests']))
    common_interest=len((interest1).intersection(interest2))
    
    #Age
    age_score= max(0,10- abs(profile1['Age']-profile2['Age']))
    
    #swipping History
    swiping_score = min(profile1['Swiping History'], profile2['Swiping History']) / 100
    
    #relationship status
    relationship_score=0
    if (profile1['Looking For']==profile2['Looking For']):
        relationship_score=1
        
    Total_score=(common_interest + age_score + swiping_score + relationship_score)
    return Total_score


profile1= male_p.iloc[8]
profile2= female_p.iloc[8]
score= rel_Score(profile1,profile2)
print(f"Match score between User {profile1['User ID']} and User {profile2['User ID']} : {score}")

Match score between User 23 and User 12 : 9.19


Recommending with use of score

In [47]:
def recommend(male_p,female_p):
    recommendation= []
    for _ , male_profile in male_p.iterrows():
        best_match=None
        best_score=-1
        
        for _ , female_profile in female_p.iterrows():
            sc=rel_Score(male_profile,female_profile)
            
            
            if sc>best_score:
                best_match=female_profile
                best_score=sc
        
        
        recommendation.append((male_profile, best_match, best_score))
    
    return recommendation


recommendation = recommend(male_p, female_p)
recommendation.sort(key=lambda x: x[2], reverse=True)

for idx, (male_profile, female_profile, score) in enumerate(recommendation[:100]):
    print(f"Recommendation {idx + 1}:")
    print(f"Male Profile (User {male_profile['User ID']}): Age {male_profile['Age']}, Interests {male_profile['Interests']}")
    print(f"Female Profile (User {female_profile['User ID']}): Age {female_profile['Age']}, Interests {female_profile['Interests']}")
    print(f"Match Score: {score}")
    print()

Recommendation 1:
Male Profile (User 36): Age 19, Interests ['Movies', 'Cooking', 'Hiking', 'Reading', 'Sports', 'Travel', 'Music']
Female Profile (User 451): Age 19, Interests ['Reading', 'Music', 'Cooking', 'Hiking', 'Travel', 'Sports', 'Movies']
Match Score: 18.79

Recommendation 2:
Male Profile (User 274): Age 29, Interests ['Reading', 'Movies', 'Travel', 'Music', 'Hiking', 'Cooking', 'Sports']
Female Profile (User 300): Age 29, Interests ['Cooking', 'Reading', 'Music', 'Hiking', 'Travel', 'Sports', 'Movies']
Match Score: 18.73

Recommendation 3:
Male Profile (User 456): Age 29, Interests ['Cooking', 'Hiking', 'Sports', 'Travel', 'Music', 'Movies', 'Reading']
Female Profile (User 65): Age 29, Interests ['Travel', 'Movies', 'Reading', 'Sports', 'Music', 'Cooking', 'Hiking']
Match Score: 18.69

Recommendation 4:
Male Profile (User 147): Age 34, Interests ['Reading', 'Travel', 'Movies', 'Hiking', 'Cooking', 'Music', 'Sports']
Female Profile (User 287): Age 34, Interests ['Reading', 'H